In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional

In [9]:
# Sample dataset

data = pd.read_csv('./NLU_Dataset.csv')

input_text = input('Enter text : ')

[('Intent', 0       Greet
1       Greet
2       Greet
3       Greet
4       Greet
       ...   
80    Inquiry
81    Inquiry
82    Inquiry
83    Inquiry
84    Inquiry
Name: Intent, Length: 85, dtype: object), ('Example', 0                              Hi there!
1                          Good morning!
2                   Hey, how's it going?
3               Hello, nice to meet you!
4                             Greetings!
                     ...                
80       Do you offer delivery services?
81        Can you recommend a good book?
82          How can I update my profile?
83    What's the price for this service?
84              Where can I park my car?
Name: Example, Length: 85, dtype: object)]


In [10]:
# Tokenization and sequence padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text for intent, examples in data.items() for text in examples])
print([text for intent, examples in data.items() for text in examples])
vocab_size = len(tokenizer.word_index) + 1

max_len = max(len(text.split()) for intent, examples in data.items() for text in examples)
sequences = tokenizer.texts_to_sequences([text for intent, examples in data.items() for text in examples])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# One-hot encode labels
labels = np.array([i for i, intent in enumerate(data.keys()) for _ in data[intent]])

['Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Greet', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Farewell', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Inquiry', 'Hi there!', 'Good morning!', "Hey, how's it going?", 'Hello, nice to meet you!', 'Greetings!', "

In [5]:
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(len(data), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
model.fit(padded_sequences, labels, epochs=50, verbose=0)

D:\My\CS\Hackothons\Intellihack\Intellihack_CodeCrafters_2\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
# Testing
test_input = "Hi there, can you tell me the time?"
test_sequence = tokenizer.texts_to_sequences([test_input])
padded_test_sequence = pad_sequences(test_sequence, maxlen=max_len, padding='post')

predicted_probabilities = model.predict(padded_test_sequence)[0]
predicted_intent_index = np.argmax(predicted_probabilities)
confidence_score = predicted_probabilities[predicted_intent_index]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step


In [7]:
# Fallback mechanism
threshold = 0.7
if confidence_score >= threshold:
    intents = list(data.keys())
    predicted_intent = intents[predicted_intent_index]
    print(f"Intent: {predicted_intent}, Confidence: {confidence_score}")
else:
    print("Fallback: Intent could not be confidently determined.")

Intent: Example, Confidence: 0.9986145496368408
